In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import os
import re
import glob
import shutil
import requests
from bs4 import BeautifulSoup

import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from PIL import Image


ROOT_PATH = os.getcwd()
HERO_NAMES_PATH = f"{ROOT_PATH}/test_data/hero_names.txt"
HERO_IMAGES_DIR = f"{ROOT_PATH}/test_data/hero_images/"
TEST_IMAGES_DIR = f"{ROOT_PATH}/test_data/test_images/"
TEST_LABELS_PATH = f"{ROOT_PATH}/test_data/test.txt"

# Get hero names list
with open(HERO_NAMES_PATH, "r") as f:
    hero_names = f.read().splitlines()
len(hero_names)

64

# Download Hero Images

In [2]:
def get_champion_hero_image_links(hero_names):
    url = "https://leagueoflegends.fandom.com/wiki/Champion_(Wild_Rift)"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    hero_image_links = {}

    for hero_name in tqdm(hero_names):
        
        # Replace spaces with underscores and fix Kai'Sa and Kha'Zix
        new_hero_name = hero_name.replace("_", " ")
        new_hero_name = "Kai'Sa" if new_hero_name == "KaiSa" else new_hero_name
        new_hero_name = "Kha'Zix" if new_hero_name == "KhaZix" else new_hero_name

        link = soup.find_all("img", attrs={'alt': new_hero_name})[0]
        
        match = re.search(r'https://.*?\.png', link["src"])

        if match:
            extracted_link = match.group(0)
        else:
            match = re.search(r'https://.*?\.png', link["data-src"])
            if match:
                extracted_link = match.group(0)
            else:
                print("No match found for ", hero_name)
                continue

        hero_image_links[hero_name] = extracted_link

    return hero_image_links

hero_image_links = get_champion_hero_image_links(hero_names)
len(hero_image_links)

100%|██████████| 64/64 [00:00<00:00, 112.68it/s]


64

In [3]:
def download_hero_images(hero_image_links, path):
    for hero_name, link in tqdm(hero_image_links.items()):
        response = requests.get(link)
        
        if response.status_code == 200:
            with open(f"{path}{hero_name}.png", "wb") as f:
                f.write(response.content)
        else:
            print(f"Failed to download image for {hero_name}.")

download_hero_images(hero_image_links, path=HERO_IMAGES_DIR)

100%|██████████| 64/64 [00:15<00:00,  4.07it/s]


In [4]:
def add_folder_each_hero_image(directory=HERO_IMAGES_DIR):
    files = os.listdir(directory)

    for file in files:
        if file.endswith('.png'):
            hero_name = file.replace('.png', '')
            os.mkdir(os.path.join(directory, hero_name))
            shutil.move(os.path.join(directory, file), os.path.join(directory, hero_name, file))

# Dataset & Dataloader

## Prepare path & label

In [3]:
test_images_path_list = glob.glob(TEST_IMAGES_DIR+"*")
print(len(test_images_path_list))

# Get labels for test images
with open(TEST_LABELS_PATH, "r") as f:
    test_labels = f.read().splitlines()
    test_file_2_labels = [label.split("\t") for label in test_labels]
    test_file_2_labels = {label[0]: label[1] for label in test_file_2_labels}

def path_2_label(path):
    file_name = path.split("\\")[-1]
    return test_file_2_labels[file_name]

path_2_label(test_images_path_list[0])

98


'Ahri'

In [4]:
hero_images_path_list = glob.glob(HERO_IMAGES_DIR+"*")
len(hero_images_path_list)

# Get labels for hero images
hero_images_path_2_label = {path: path.split("\\")[-1].split(".")[0] for path in hero_images_path_list}

In [5]:
hero_images_path_2_label

{'d:\\AI_Engineer_Test/test_data/hero_images\\Ahri.png': 'Ahri',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Akali.png': 'Akali',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Alistar.png': 'Alistar',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Amumu.png': 'Amumu',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Annie.png': 'Annie',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Ashe.png': 'Ashe',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Aurelion_Sol.png': 'Aurelion_Sol',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Blitzcrank.png': 'Blitzcrank',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Braum.png': 'Braum',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Camille.png': 'Camille',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Corki.png': 'Corki',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Darius.png': 'Darius',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Diana.png': 'Diana',
 'd:\\AI_Engineer_Test/test_data/hero_images\\Dr._Mundo.png': 'Dr',
 'd:\\AI_Engineer_Test/test_dat

## Transform 

In [6]:
import random
from torchvision.transforms import transforms
import torch.nn.functional as F
import torchvision.transforms.functional as TF


class RandomTransform:
    def __init__(self, p=0.5, max_padding=100, noise=0.1):
        self.p = p
        self.max_padding = max_padding
        self.noise = noise
        
    def __call__(self, x):
        if random.random() < self.p:
            # Crop, Rotate, ColorJitter
            x = TF.rotate(x, angle=random.randint(-45, 45))
            x = TF.resized_crop(x, top=random.randint(0, 50), left=random.randint(0, 50), height=256, width=256, size=(256, 256))

            # Random padding
            padding = random.randint(0, self.max_padding)
            x = TF.pad(x, padding, padding_mode='reflect')
            
            # Pad with random colors
            padding_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            x = TF.pad(x, padding, padding_mode='constant', fill=padding_color)
            
            # Adjust brightness, contrast, and saturation
            brightness = random.uniform(0.7, 1.2)
            contrast = random.uniform(0.7, 1.2)
            saturation = random.uniform(0.7, 1.2)
            x = TF.adjust_brightness(x, brightness_factor=brightness)
            x = TF.adjust_contrast(x, contrast_factor=contrast)
            x = TF.adjust_saturation(x, saturation_factor=saturation)
            
            # Add blender nois
            x = TF.gaussian_blur(x, kernel_size=5)
            img_blended = Image.new(x.mode, x.size)
            x = Image.blend(x, img_blended, alpha=random.uniform(0.1, 0.5))
                                
        return x

In [9]:
transform = transforms.Compose([
                transforms.Resize((256, 256)),
                RandomTransform(),
                transforms.Resize((256, 256)),
                # transforms.ToTensor(),
                # transforms.Normalize(mean=(0.5, 0.5, 0.5), 
                #                      std=(0.5, 0.5, 0.5))
            ])

# image = Image.open(test_images_path_list[70])
image = Image.open(hero_images_path_list[9])
transformed_image = transform(image)
transformed_image.show()

## Dataset

In [7]:
from torch.utils.data import Dataset
from tqdm import tqdm


class HeroImagesDataset(Dataset):
    def __init__(self, 
                 hero_images_path_list, 
                 hero_images_path_2_label,
                 num_triplets=20,
                 train=True,
                 to_tensor=True):
        
        self.hero_images_path_list = hero_images_path_list
        self.hero_images_path_2_label = hero_images_path_2_label
        self.num_triplets = num_triplets
        self.train = train
        self.to_tensor = to_tensor

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            RandomTransform(p=0.7),
            transforms.Resize((256, 256)),
        ])

        if self.to_tensor:
            self.transform_to_tensor = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225]),
            ])

        self.data, self.target = self.load_data()

    def load_data(self):
        data_samples = []
        targets = []

        for anchor_img_path in tqdm(self.hero_images_path_list):
            hero_label = self.hero_images_path_2_label[anchor_img_path]

            triplet_samples = self.create_triplet_samples(anchor_img_path)

            data_samples += triplet_samples
            targets += [hero_label] * len(triplet_samples)

        return data_samples, targets

    def create_triplet_samples(self, anchor_img_path):
        triplet_samples = []

        neg_img_paths = random.sample([p for p in self.hero_images_path_list if p != anchor_img_path], self.num_triplets)
        
        for neg_img_path in neg_img_paths:
            anchor_img = Image.open(anchor_img_path).convert('RGB')
            pos_img = Image.open(anchor_img_path).convert('RGB')
            neg_img = Image.open(neg_img_path).convert('RGB')

            # Apply transformations for positive and negative images
            pos_img = self.transform(pos_img)
            neg_img = self.transform(neg_img)

            # Apply transformations to tensor
            if self.to_tensor:
                anchor_img = self.transform_to_tensor(anchor_img)
                pos_img = self.transform_to_tensor(pos_img)
                neg_img = self.transform_to_tensor(neg_img)

            triplet_samples.append((anchor_img, pos_img, neg_img))

        return triplet_samples
    
    def __getitem__(self, index):
        return self.data[index], self.target[index]
    
    def __len__(self):
        return len(self.data)


# Inference and Get top 20 retrieval

In [7]:
# Define the model to be used for feature extraction
model = torchvision.models.resnet18(pretrained=False)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the last layer (classifier)

ckpt_path = "D:\AI_Engineer_Test\HeroDetection\logs\\train\\runs\\2023-03-31_13-05-21\checkpoints\epoch_021.ckpt"
checkpoint = torch.load(ckpt_path)
ckpt_state_dict = {key.replace('model.', ''): value for key, value in checkpoint['state_dict'].items()}

model.load_state_dict(ckpt_state_dict)
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [8]:
class CircleCrop(object):
    def __init__(self, size):
        self.size = size
        
    def __call__(self, img):
        # Convert PIL image to PyTorch tensor
        img = transforms.ToTensor()(img)
        
        # Define circular mask
        mask = np.zeros((img.shape[-2], img.shape[-1]))
        center = [img.shape[-1] / 2, img.shape[-2] / 2]
        radius = min(img.shape[-1], img.shape[-2]) / 2
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if (i - center[1]) ** 2 + (j - center[0]) ** 2 <= radius ** 2:
                    mask[i, j] = 1
        
        # Apply mask to image tensor
        img = img * mask
        
        # Convert tensor back to PIL image
        img = transforms.ToPILImage()(img)
        
        # Apply additional transforms
        transform = transforms.Compose([
            transforms.Resize(self.size),
            transforms.CenterCrop(self.size)
        ])
        img = transform(img)
        
        return img


In [9]:
def get_embedding(model, image_path, is_test=True, downsize=40):
    if is_test:
        transform = transforms.Compose([
                        transforms.Lambda(lambda x: x.crop((0, 0, int(x.height*1.2), x.height))),  # Crop the left side
                        transforms.Resize((256, 256)),
                        transforms.CenterCrop(175),
                        # CircleCrop(size=200),  # Crop the left side
                        # transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                            std=[0.229, 0.224, 0.225]),
                    ])
    else:
        transform = transforms.Compose([
                        transforms.Resize((downsize, downsize)),
                        transforms.Resize((256, 256)),
                        CircleCrop(size=256),  # Crop the left side
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                            std=[0.229, 0.224, 0.225]),
                    ])
    
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)

    with torch.no_grad():
        embedding = model(image).squeeze().cpu().detach().numpy()
        return embedding

In [10]:
import annoy

n_trees = 100  # Number of trees in the index

annoy_index = annoy.AnnoyIndex(f=512, metric='euclidean')
num_heroes = len(hero_images_path_list)

for k, downsize in enumerate([25, 30, 35]):
    for i, hero_path in tqdm(enumerate(hero_images_path_list), total=num_heroes):
        embedding = get_embedding(model, hero_path, is_test=False, downsize=downsize)
        annoy_index.add_item(i+k*num_heroes, embedding)

annoy_index.build(n_trees)
annoy_index.get_n_items() 


100%|██████████| 64/64 [00:07<00:00,  8.68it/s]


192

In [105]:
transform = transforms.Compose([
                        transforms.Lambda(lambda x: x.crop((0, 0, int(x.height*1), x.height))),  # Crop the left side
                        transforms.Resize((40, 40)),
                        transforms.Resize((256, 256)),
                        # transforms.CenterCrop(180),
                        CircleCrop(size=256),  # Crop the left side
                        transforms.Resize((256, 256)),
                        # transforms.ToTensor(),
                        # transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        #                     std=[0.229, 0.224, 0.225]),
                    ])
image = Image.open(hero_images_path_list[11]).convert('RGB')
image = transform(image)
image.show()

In [11]:
k = 10  # Number of nearest neighbors to retrieve

query_path = test_images_path_list[10]
# query_path = hero_images_path_list[35]
print(path_2_label(query_path))
# print(query_path)
query_embedding = get_embedding(model, query_path, is_test=True)

nn_indices, nn_scores = annoy_index.get_nns_by_vector(query_embedding, k, include_distances=True)
print(annoy_index.get_nns_by_vector(query_embedding, k, include_distances=True))
nn_indices = [list(hero_images_path_2_label.values())[i%num_heroes] for i in nn_indices]
nn_indices

Akali
([1, 65, 129, 39, 170, 127, 106, 103, 10, 63], [15.045661926269531, 15.535541534423828, 15.846200942993164, 17.097822189331055, 17.5467472076416, 17.57960319519043, 17.586578369140625, 17.600475311279297, 17.613298416137695, 17.630596160888672])


['Akali',
 'Akali',
 'Akali',
 'Nami',
 'Orianna',
 'Ziggs',
 'Orianna',
 'Nami',
 'Corki',
 'Ziggs']

In [18]:
acc = 0
k=3

for test_path in tqdm(test_images_path_list):
    query_label = path_2_label(test_path)
    query_embedding = get_embedding(model, test_path, is_test=True)
    nn_indices, nn_scores = annoy_index.get_nns_by_vector(query_embedding, k, include_distances=True)
    nn_labels = [list(hero_images_path_2_label.values())[i%64] for i in nn_indices]
     
    if query_label in nn_labels:
        acc += 1

print(f"Accuracy: {acc/len(test_images_path_list)}")

100%|██████████| 98/98 [00:02<00:00, 35.29it/s]

Accuracy: 0.7448979591836735
